In [2]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Short Let

In [4]:
import datetime
yesterday = datetime.date.today() - datetime.timedelta(days=+1)
yesterday


datetime.date(2023, 9, 30)

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import concurrent.futures
import datetime

today = datetime.date.today()
yesterday = datetime.date.today() - datetime.timedelta(days=+1)
today = '_' + str(today)

url = [f'https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page={i:d}'  for i in (range(0, 200))]
titles= []
types = []
locations = []
prices = []
date_posted = []
PIDs = []
furnished = []
beds = []
agents = []


def extract_data(url):
    print('began')
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text,  "html.parser")
    house_box = soup.find_all('div', class_ = "single-room-sale listings-property")
    for house in house_box:
#titles
        if house.find('h3', class_ = "listings-property-title2") is not None:
            title = house.find('h3', class_ = "listings-property-title2").text
            titles.append(title)
        else:
            titles.append('No title')

#types
        if house.find('h4', class_ = "listings-property-title") is not None:
            type = house.find('h4', class_ = "listings-property-title").text
            types.append(type)
        else:
            types.append('No type')

#locations
        if house.find('h4') is not None:
            locate = house.find_all('h4')
            location = locate[1].text
            locations.append(location)
        else:
            locations.append('No location')

#prices
        if house.find('h3', class_ = "listings-price") is not None:
            price = house.find('h3', class_ = "listings-price").text
            prices.append(price)
        else:
            prices.append('No price')

#date_posted
        if house.find('h5') is not None:
            date = house.find('h5').text
            date_posted.append(date)
        else:
            date_posted.append('No date')

#PIDs
        if house.find('h2') is not None:
            PID = house.find('h2').text.replace('PID:','')
            PIDs.append(PID)
        else:
            PIDs.append('No PID')

#furnished, serviced, newly built
        if house.find('div', class_ = "furnished-btn") is not None:
            furnish = house.find('div', class_ = "furnished-btn").text
            furnished.append(furnish)
        else:
            furnished.append('0')

#utilities
        if house.find('div', class_ = "fur-areea") is not None:
            bed = house.find('div', class_= "fur-areea").text.replace('\n',' ').strip()
            beds.append(bed)
        else:
            beds.append('No beds')
        
#agents
        if house.find('div', class_ = "elite-icon") is not None:
            agent = house.find('div', class_ = "elite-icon").a.get('href')
            agent = agent.replace('/agent/','')
            agents.append(agent)
        else:
            agents.append('No agent')


def transform_data():
    df = pd.DataFrame({'title': titles, 
                            'categories': types,
                            'address': locations,
                            'agent': agents,
                            'price': prices,
                            'date_post': date_posted,
                            'PIDs': PIDs,
                            'furnish': furnished,
                            'bed': beds})

    df['newly_built'] = df['furnish'].apply(lambda text: 'Newly Built' in text)
    df['serviced'] = df['furnish'].apply(lambda text: 'Serviced' in text)
    df['furnished'] = df['furnish'].apply(lambda text: 'Furnished' in text)
    df.drop('furnish', axis=1, inplace=True)
    df[['beds', 'baths', 'toilets']] = df['bed'].str.extract(r'(\d+)\s*beds?\s*(\d*)\s*baths?\s*(\d*)\s*Toilets?')
    df['beds'] = pd.to_numeric(df['beds'], errors='coerce').fillna(0).astype(int)
    df['baths'] = pd.to_numeric(df['baths'], errors='coerce').fillna(0).astype(int)
    df['toilets'] = pd.to_numeric(df['toilets'], errors='coerce').fillna(0).astype(int)

    df['price'] = df['price'].str.replace('₦', '')

    df['price_₦_yearly'] = pd.to_numeric(df['price'].str.replace(',', '').str.extract(r'(\d+)')[0])

    
    df.drop('price', axis=1, inplace=True)
    df.drop('bed', axis=1, inplace=True)

    df['date_posted'] = df['date_post'].str.extract(r'Added (\d{2} \w{3} \d{4})', expand=False)
    df['date_updated'] = df['date_post'].str.extract(r'Updated (\d{2} \w{3} \d{4})', expand=False)
    df['date_posted'] = pd.to_datetime(df['date_posted'], format='%d %b %Y', errors='coerce')
    df['date_updated'] = pd.to_datetime(df['date_updated'], format='%d %b %Y', errors='coerce')
    #df['date_updated'] = df['date_updated'].fillna("not updated")
    df.drop('date_post', axis=1, inplace=True)
    df['state'] = df['address'].str.split().str[-1]
    df = df[(df['date_posted'].dt.date == yesterday) | (df['date_updated'].dt.date == yesterday)]
    
    
    df.to_csv(f'../Real_Estate_data_pipeline/property_csv/propertypro_for_rent{yesterday}.csv', index=False)
    return df

    

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(extract_data, url)

transform_data()


began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=0
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=1
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=2
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=3
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=4
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=5
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=6
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=7
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=8
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=9
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=10
began
https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page=11
be

## For sale

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import concurrent.futures


url = [f'https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page={i:d}'  for i in (range(0, 865))]
titles= []
types = []
locations = []
prices = []
date_posted = []
PIDs = []
furnished = []
beds = []
agents = []


def extract_data(url):
    print('began')
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text,  "html.parser")
    house_box = soup.find_all('div', class_ = "single-room-sale listings-property")
    for house in house_box:
#titles
        if house.find('h3', class_ = "listings-property-title2") is not None:
            title = house.find('h3', class_ = "listings-property-title2").text
            titles.append(title)
        else:
            titles.append('No title')

#types
        if house.find('h4', class_ = "listings-property-title") is not None:
            type = house.find('h4', class_ = "listings-property-title").text
            types.append(type)
        else:
            types.append('No type')

#locations
        if house.find('h4') is not None:
            locate = house.find_all('h4')
            location = locate[1].text
            locations.append(location)
        else:
            locations.append('No location')

#prices
        if house.find('h3', class_ = "listings-price") is not None:
            price = house.find('h3', class_ = "listings-price").text
            prices.append(price)
        else:
            prices.append('No price')

#date_posted
        if house.find('h5') is not None:
            date = house.find('h5').text
            date_posted.append(date)
        else:
            date_posted.append('No date')

#PIDs
        if house.find('h2') is not None:
            PID = house.find('h2').text.replace('PID:','')
            PIDs.append(PID)
        else:
            PIDs.append('No PID')

#furnished, serviced, newly built
        if house.find('div', class_ = "furnished-btn") is not None:
            furnish = house.find('div', class_ = "furnished-btn").text
            furnished.append(furnish)
        else:
            furnished.append('0')

#utilities
        if house.find('div', class_ = "fur-areea") is not None:
            bed = house.find('div', class_= "fur-areea").text.replace('\n',' ').strip()
            beds.append(bed)
        else:
            beds.append('No beds')
        
#agents
        if house.find('div', class_ = "elite-icon") is not None:
            agent = house.find('div', class_ = "elite-icon").a.get('href')
            agent = agent.replace('/agent/','')
            agents.append(agent)
        else:
            agents.append('No agent')


def transform_data():
    df = pd.DataFrame({'title': titles, 
                            'categories': types,
                            'address': locations,
                            'agent': agents,
                            'price': prices,
                            'date_post': date_posted,
                            'PIDs': PIDs,
                            'furnish': furnished,
                            'bed': beds})

    df['newly_built'] = df['furnish'].apply(lambda text: 'Newly Built' in text)
    df['serviced'] = df['furnish'].apply(lambda text: 'Serviced' in text)
    df['furnished'] = df['furnish'].apply(lambda text: 'Furnished' in text)
    df.drop('furnish', axis=1, inplace=True)
    df[['beds', 'baths', 'toilets']] = df['bed'].str.extract(r'(\d+)\s*beds?\s*(\d*)\s*baths?\s*(\d*)\s*Toilets?')
    df['beds'] = pd.to_numeric(df['beds'], errors='coerce').fillna(0).astype(int)
    df['baths'] = pd.to_numeric(df['baths'], errors='coerce').fillna(0).astype(int)
    df['toilets'] = pd.to_numeric(df['toilets'], errors='coerce').fillna(0).astype(int)

    df['price'] = df['price'].str.replace('₦', '')

    df['price_₦'] = pd.to_numeric(df['price'].str.replace(',', '').str.extract(r'(\d+)')[0])

    
    df.drop('price', axis=1, inplace=True)
    df.drop('bed', axis=1, inplace=True)

    df['date_posted'] = df['date_post'].str.extract(r'Added (\d{2} \w{3} \d{4})', expand=False)
    df['date_updated'] = df['date_post'].str.extract(r'Updated (\d{2} \w{3} \d{4})', expand=False)
    df['date_posted'] = pd.to_datetime(df['date_posted'], format='%d %b %Y')
    df['date_updated'] = pd.to_datetime(df['date_updated'], format='%d %b %Y')
    df['date_updated'] = df['date_updated'].fillna("not updated")
    df.drop('date_post', axis=1, inplace=True)
    df['state'] = df['address'].str.split().str[-1]
    df.to_csv('propertypro_for_sale.csv', index=False)
    

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(extract_data, url)

transform_data()


began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=0
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=1
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=2
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=3
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=4
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=5
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=6
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=7
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=8
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=9
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=10
began
https://www.propertypro.ng/property-for-sale?sort=postedOn&order=desc&page=11
be

## For Rent

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import concurrent.futures

url = [f'https://www.propertypro.ng/property-for-rent?sort=postedOn&order=desc&page={i:d}'  for i in (range(0, 424))]
titles= []
types = []
locations = []
prices = []
date_posted = []
PIDs = []
furnished = []
beds = []
agents = []


def extract_data(url):
    print('began')
    page = requests.get(url)
    soup = BeautifulSoup(page.text,  "html.parser")
    house_box = soup.find_all('div', class_ = "single-room-sale listings-property")
    for house in house_box:
#titles
        if house.find('h3', class_ = "listings-property-title2") is not None:
            title = house.find('h3', class_ = "listings-property-title2").text
            titles.append(title)
        else:
            titles.append('No title')

#types
        if house.find('h4', class_ = "listings-property-title") is not None:
            type = house.find('h4', class_ = "listings-property-title").text
            types.append(type)
        else:
            types.append('No type')

#locations
        if house.find('h4') is not None:
            locate = house.find_all('h4')
            location = locate[1].text
            locations.append(location)
        else:
            locations.append('No location')

#prices
        if house.find('h3', class_ = "listings-price") is not None:
            price = house.find('h3', class_ = "listings-price").text
            prices.append(price)
        else:
            prices.append('No price')

#date_posted
        if house.find('h5') is not None:
            date = house.find('h5').text
            date_posted.append(date)
        else:
            date_posted.append('No date')

#PIDs
        if house.find('h2') is not None:
            PID = house.find('h2').text.replace('PID:','')
            PIDs.append(PID)
        else:
            PIDs.append('No PID')

#furnished, serviced, newly built
        if house.find('div', class_ = "furnished-btn") is not None:
            furnish = house.find('div', class_ = "furnished-btn").text
            furnished.append(furnish)
        else:
            furnished.append('0')

#utilities
        if house.find('div', class_ = "fur-areea") is not None:
            bed = house.find('div', class_= "fur-areea").text.replace('\n',' ').strip()
            beds.append(bed)
        else:
            beds.append('No beds')
        
#agents
        if house.find('div', class_ = "elite-icon") is not None:
            agent = house.find('div', class_ = "elite-icon").a.get('href')
            agent = agent.replace('/agent/','')
            agents.append(agent)
        else:
            agents.append('No agent')


def transform_data():
    df = pd.DataFrame({'title': titles, 
                            'categories': types,
                            'address': locations,
                            'agent': agents,
                            'price': prices,
                            'date_post': date_posted,
                            'PIDs': PIDs,
                            'furnish': furnished,
                            'bed': beds})

    df['newly_built'] = df['furnish'].apply(lambda text: 'Newly Built' in text)
    df['serviced'] = df['furnish'].apply(lambda text: 'Serviced' in text)
    df['furnished'] = df['furnish'].apply(lambda text: 'Furnished' in text)
    df.drop('furnish', axis=1, inplace=True)
    df[['beds', 'baths', 'toilets']] = df['bed'].str.extract(r'(\d+)\s*beds?\s*(\d*)\s*baths?\s*(\d*)\s*Toilets?')
    df['beds'] = pd.to_numeric(df['beds'], errors='coerce').fillna(0).astype(int)
    df['baths'] = pd.to_numeric(df['baths'], errors='coerce').fillna(0).astype(int)
    df['toilets'] = pd.to_numeric(df['toilets'], errors='coerce').fillna(0).astype(int)

    df['price'] = df['price'].str.replace('₦', '')

    df['price_₦_yearly'] = pd.to_numeric(df['price'].str.replace(',', '').str.extract(r'(\d+)')[0])

    
    df.drop('price', axis=1, inplace=True)
    df.drop('bed', axis=1, inplace=True)

    df['date_posted'] = df['date_post'].str.extract(r'Added (\d{2} \w{3} \d{4})', expand=False)
    df['date_updated'] = df['date_post'].str.extract(r'Updated (\d{2} \w{3} \d{4})', expand=False)
    df['date_posted'] = pd.to_datetime(df['date_posted'], format='%d %b %Y')
    df['date_updated'] = pd.to_datetime(df['date_updated'], format='%d %b %Y')
    df['date_updated'] = df['date_updated'].fillna("not updated")
    df.drop('date_post', axis=1, inplace=True)
    df['state'] = df['address'].str.split().str[-1]
    df.to_csv('propertypro_for_rent.csv', index=False)
    return df
    

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(extract_data, url)

transform_data()


# Agents

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import concurrent.futures
import datetime
today = datetime.date.today()
today = '_' + str(today)
url = [f'https://www.propertypro.ng/agents?page={i:d}'  for i in (range(0, 2650))]

names = []
locations = []
registered = []



def extract_data(url):
    print('began')
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text,  "html.parser")
    print(soup)
    house_box = soup.find_all('div', class_ = "agent-rp-inner")
    for house in house_box:
#names
        if house.find('div', class_ = "rp-left-text") is not None:
            name = house.find('div', class_ = "rp-left-text").h2.text.replace('\n','')
            names.append(name)
        else:
            names.append('No name')

        if house.find('div', class_ = "rp-left-text") is not None:
            locate = house.find('div', class_ = "rp-left-text").h4.text
            locations.append(locate)
        else:
            locations.append('None')

        if house.find('div', class_ = "rp-left-text") is not None:
            register = house.find('div', class_ = "rp-left-text").text.replace('\n','')
            register = register[register.find('Registered on'):]
            registered.append(register)
        else:
            registered.append('None')
print(names)

def transform_data():
    df = pd.DataFrame({'name': names, 
                        'located': locations,
                        'registered_on': registered
                        })
    print(df)
    df['registered_on'] = df['registered_on'].replace('Registered on ', '')
    df['registered_on'] = df['registered_on'].replace('agent', '')
    df.to_csv('../Real_Estate_data_pipeline_NG/property_csv/agents_staging.csv',index=False)

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(extract_data, url)

transform_data()

[]
began
https://www.propertypro.ng/agents?page=0
began
https://www.propertypro.ng/agents?page=1
began
https://www.propertypro.ng/agents?page=2
began
https://www.propertypro.ng/agents?page=3
began
https://www.propertypro.ng/agents?page=4
began
https://www.propertypro.ng/agents?page=5
began
https://www.propertypro.ng/agents?page=6
began
https://www.propertypro.ng/agents?page=7
began
https://www.propertypro.ng/agents?page=8
began
https://www.propertypro.ng/agents?page=9
began
https://www.propertypro.ng/agents?page=10
began
https://www.propertypro.ng/agents?page=11
began
https://www.propertypro.ng/agents?page=12
began
https://www.propertypro.ng/agents?page=13
began
https://www.propertypro.ng/agents?page=14
began
https://www.propertypro.ng/agents?page=15
began
https://www.propertypro.ng/agents?page=16
began
https://www.propertypro.ng/agents?page=17
began
https://www.propertypro.ng/agents?page=18
began
https://www.propertypro.ng/agents?page=19



<!DOCTYPE html>

<html lang="en">
<head>
<title>Real Estate Agents &amp; Companies in Nigeria | PropertyPro Nigeria</title>
<script>
                (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start': new Date().getTime(), event:'gtm.js'});
                    var f=d.getElementsByTagName(s)[0],j=d.createElement(s),
                            dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src= 'https://www.googletagmanager.com/gtm.js?id='+i+dl;
                            f.parentNode.insertBefore(j,f);
                })(window,document,'script','dataLayer','GTM-TW3LXRX');
        </script>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/assets/image/42484569b600758112d45bad3d876f78-favicon.ico" rel="icon" type="image/ico"/>
<meta content="Property Consultants, Developers, Brokers, Surveyors, Real estate businesses, Realtors and Real Estate Agents &amp; Companies" name="description">
<meta content="Property Consultants, 

In [6]:
#pip install psycopg2
! pip install sqlalchemy

  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/54/c2/c51f040038859732f781f25907e01ee980987d24fa0747884e6073363a14/SQLAlchemy-2.0.21-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached SQLAlchemy-2.0.21-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.4 kB)
  Obtaining dependency information for greenlet!=0.4.17 from https://files.pythonhosted.org/packages/6c/df/1e3e52e35e56b912c7bcd64ba2010d6972c43dff96794074b32a62345970/greenlet-3.0.0-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata
Using cached SQLAlchemy-2.0.21-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.4/616.4 kB 12.8 MB/s eta 0:00:00a 0:00:01


In [5]:
def transform_data():
    df = pd.DataFrame({'name': names, 
                        'located': locations,
                        'registered_on': registered
                        })
    df['registered_on'] = df['registered_on'].replace('Registered on ', '')
    df['registered_on'] = df['registered_on'].replace('agent', '')
    df.to_csv('../Real_Estate_data_pipeline_NG/property_csv/agents_staging.csv',index=False)

transform_data()